In [34]:
import json
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext, sql
spark = SparkSession.builder.appName('Top 10 Restaurants').getOrCreate()

In [35]:
sqlContext = SQLContext(sparkContext=spark.sparkContext, sparkSession=spark)
# query = 'select * from restaurant_basic_info where start >= '
source_df = sqlContext.read.format('jdbc').options(
    url='jdbc:mysql://dsci551-project.cjy5mlvqnx2k.us-west-1.rds.amazonaws.com:3306/restaurants_info',
    driver='com.mysql.jdbc.Driver', #com.mysql.jdbc.Driver
    dbtable="(select * from restaurant_basic_info where stars >= '3.0') restaurant_basic_info",
    user='admin',
    password='password').load()

In [36]:
category_list = [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
vector_df = sqlContext.read.format('jdbc').options(
    url='jdbc:mysql://dsci551-project.cjy5mlvqnx2k.us-west-1.rds.amazonaws.com:3306/restaurants_info',
    driver='com.mysql.jdbc.Driver', #com.mysql.jdbc.Driver
    dbtable="restaurant_vector",
    user='admin',
    password='password').load()

In [37]:
vector_df = vector_df.join(source_df, on=['business_id'], how='left_semi')

In [39]:
def dot_product(x):
    x = map(int, x.split(','))
    return sum(i[0] * i[1] for i in zip(x, category_list))
dot_product_df = vector_df.rdd.map(lambda row : (row['business_id'], dot_product(row["binary_vector"]))).toDF(["business_id", "scores"])

In [40]:
dot_product_df = dot_product_df.sort('scores', ascending=False).limit(10)

In [41]:
source_df = source_df.join(dot_product_df, on=['business_id'], how='right')

In [43]:
results = source_df[["name", "address", "state", "city", "categories", "stars", "scores"]].toJSON().map(lambda j: json.loads(j)).collect()
print(results)

[{'name': 'Yuki Japanese & Korean Restaurant', 'address': '3259 Dufferin St', 'state': 'ON', 'city': 'North York', 'categories': 'Japanese, Restaurants, Korean', 'stars': '3.5', 'scores': 1}, {'name': 'Sushi Ramen GO', 'address': '1495 S Higley Rd, Ste 104', 'state': 'AZ', 'city': 'Gilbert', 'categories': 'Sushi Bars, Ramen, Japanese, Restaurants', 'stars': '4.5', 'scores': 1}, {'name': 'Kaizoku', 'address': '6461 Old Monroe Rd', 'state': 'NC', 'city': 'Indian Trail', 'categories': 'Japanese, Poke, Restaurants, Food', 'stars': '4.0', 'scores': 1}, {'name': 'Sushi Bae', 'address': '4910A Rue Wellington', 'state': 'QC', 'city': 'Montréal', 'categories': 'Restaurants, Japanese, Sushi Bars', 'stars': '4.0', 'scores': 1}, {'name': 'The Ka', 'address': '2269 N Green Valley Pkwy', 'state': 'NV', 'city': 'Las Vegas', 'categories': 'Restaurants, Japanese, Sushi Bars, Asian Fusion', 'stars': '3.0', 'scores': 1}, {'name': 'Hibachi Express', 'address': '305 Herlong Ave', 'state': 'SC', 'city': 'Ro